In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.rmtree('/content/drive/MyDrive/yolo_custom_model_Training3')

In [ ]:
!nvidia-smi -L


In [ ]:
%cd /content/drive/MyDrive
!mkdir yolo_custom_model_Training3

In [ ]:
!ls '/content/drive/MyDrive/yolo_custom_model_Training3'


In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3
!mkdir custom_data

In [ ]:
!ls '/content/drive/MyDrive/yolo_custom_model_Training3'


In [ ]:
!unzip '/content/drive/MyDrive/yolo_custom_model_Training3/kickboard.zip' -d '/content/drive/MyDrive/yolo_custom_model_Training3/custom_data'


In [ ]:
%pwd


In [ ]:
# download dataset in current directory(above path)
!git clone 'https://github.com/AlexeyAB/darknet.git' '/content/drive/MyDrive/yolo_custom_model_Training3/darknet'

In [ ]:
# move current directory to darknet
%cd /content/drive/MyDrive/yolo_custom_model_Training3/darknet

In [ ]:
# change setting values in 'Makefile' file 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# Compile model
"""  take care do not disconnect : file directory may be interupted 
if your network down during compile, I recommend delete darknet folder and restart number 4(get AlexeyAB/darknet)"""

!make

In [ ]:
%cd ..
!darknet/darknet

In [ ]:
!git clone 'https://github.com/jakkcoder/training_yolo_custom_object_detection_files' '/content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main'

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main

In [ ]:
# check out current dir files
!ls

In [ ]:
# copy creating-train-and-test-txt-files.py & creating-files-data-and-name.py
"""creating-train-and-test-txt-files.py >> create 'train.txt' & 'test.txt' files
   creating-files-data-and-name.py >> create label 'labelled_data.data' file
   <if you excute both .py files, you get mentioned files upper lines 2,3>"""

!cp creating-train-and-test-txt-files.py /content/drive/MyDrive/yolo_custom_model_Training3//custom_data
!cp creating-files-data-and-name.py /content/drive/MyDrive/yolo_custom_model_Training3/custom_data

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/custom_data

In [ ]:
# change paths in both .py files
!sed -i '39 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-train-and-test-txt-files.py
!sed -i '74 s@jpeg@jpg@' creating-train-and-test-txt-files.py
!sed -i '36 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-files-data-and-name.py

In [ ]:
# move current dir one step before
%cd ..

In [ ]:
# excute .py file >> 'train.txt', 'test.txt'   
!python custom_data/creating-train-and-test-txt-files.py

In [ ]:
# excute .py file >> 'labelled_data.data'
!python custom_data/creating-files-data-and-name.py

In [ ]:
# create directory 'custom_weight'
!mkdir custom_weight

In [ ]:
%pwd


In [ ]:
%ls -l

In [ ]:
# move 'yolov4.conv.137' file to 'custom_weight' dir
!mv yolov4.conv.137 custom_weight/

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg


In [ ]:
# copy yolov4.cfg file & rename & paste
!cp yolov4.cfg yolov4_custom.cfg

In [ ]:
# change values for training
!sed -i '2 s@batch=64@batch=8@' yolov4_custom.cfg

!sed -i '7 s@width=608@width=416@' yolov4_custom.cfg
!sed -i '8 s@height=608@height=416@' yolov4_custom.cfg  

!sed -i '19 s@500500@2000@' yolov4_custom.cfg  #maxbatch=class*2000
!sed -i '21 s@400000,450000@1600,1800@' yolov4_custom.cfg  #maxbatch*0.8, maxbatch*0.9

!sed -i '968 s@classes=80@classes=1@' yolov4_custom.cfg
!sed -i '1056 s@classes=80@classes=1@' yolov4_custom.cfg
!sed -i '1144 s@classes=80@classes=1@' yolov4_custom.cfg

!sed -i '961 s@filters=255@filters=18@' yolov4_custom.cfg  #filters=(4+1+classes)*3 
!sed -i '1049 s@filters=255@filters=18@' yolov4_custom.cfg
!sed -i '1137 s@filters=255@filters=18@' yolov4_custom.cfg

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3


In [ ]:
!mkdir backup


In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3


In [ ]:
!darknet/darknet detector train custom_data/labelled_data.data darknet/cfg/yolov4_custom.cfg custom_weight/yolov4.conv.137 -dont_show


In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  #plt.show('')

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3
!ls


In [ ]:
# only works if the training does not get interrupted 
imShow('chart.png')

In [ ]:
%cd custom_data

!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training3/custom_data@' test.txt
!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training3/custom_data@' train.txt

!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training3/custom_data@' labelled_data.data
!sed -i '5 s@.*@backup = /content/drive/MyDrive/yolo_custom_model_Training3/backup/@' labelled_data.data

!cat labelled_data.data

In [ ]:
%cd /content/drive/My Drive/yolo_custom_model_Training3/darknet
!chmod +x ./darknet

In [ ]:
#You can check the mAP for all the saved weights to see which gives the best results

!./darknet detector map /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training3/backup/yolov4_custom_final.weights -points 0

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training3/backup/yolov4_custom_final.weights /content/drive/MyDrive/yolo_custom_model_Training3/testing/kick1.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training3/backup/yolov4_custom_final.weights /content/drive/MyDrive/yolo_custom_model_Training3/testing/kick2.png -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training3/backup/yolov4_custom_final.weights /content/drive/MyDrive/yolo_custom_model_Training3/testing/kick3.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training3/backup/yolov4_custom_final.weights /content/drive/MyDrive/yolo_custom_model_Training3/testing/kick4.jpg -thresh 0.3
imShow('predictions.jpg')